# Estimación de Valoración y Recomendación de Juegos en Steam  

**Grupo 6**  
- **Rodrigo Juanes**  
- Noé López  
- Marc Velasco  
- Pablo Selma  
- Carlos Ribes

---

### Descripción del Proyecto  
Este proyecto tiene como objetivo:
1. **Obtener reseñas de Steam** mediante técnicas de web scrapping.
2. **Procesar las reseñas**  para el correcto entrenamiento de los siguientes modelos.
1. **Determinar si un usuario ha recomendado un juego** basándose en el análisis de su opinión.  
2. **Sugerir juegos relacionados** a partir de un texto escrito, empleando las etiquetas de género y características de los juegos.  
3. **Detectar ironía en las reseñas**, para mejorar la interpretación de las valoraciones.  

---

### Tecnologías y Recursos  
- **WS/API:** SteamDB, AppReviews

## Requirements

Librerías necesarias para trabajar con el notebook.

In [1]:
!pip install -r "requirements.txt"


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


## Carga de librerías y funciones

In [2]:
# Librerías necesarias
import spacy
import numpy as np
import requests
import pandas as pd
import re
import string
import time

# Funiones necesarias
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
from unidecode import unidecode
from stop_words import get_stop_words

In [3]:
!python -m spacy download es_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 56.3 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')


# Web scrapping

El objetivo es realizar el estudio sobre una gran variedad de reviews. La mejor manera de obtener esta gran cantidad de datos es mediante *web scrapping*. Dividiremos nuestro *web scrapping* en dos partes.

* **Extracción de reseñas**
* **Obtener una lista de juegos**

## Extracción de reseñas.

Empezaremos nuestro *web scrapping* estudiando como extraer reviews de un juego concreto. Como ejemplo, usaremos el videojuego Dota 2.

El link asociado a las primeras $n$ reseñas del juego de steam con $id = \text{value}$ en lenguaje español es:
- https://store.steampowered.com/appreviews/value?json=1&num_per_page=n&language=spanish

A partir de esta API podemos obtener fácilmente las reseñas de cualquier juego.

In [4]:
# URL del endpoint de reseñas
url = 'https://store.steampowered.com/appreviews/570?json=1&num_per_page=100&language=spanish'

# Realiza la petición
response = requests.get(url)
data = response.json() 

# Itera por las reseñas
for review in data['reviews']:
    print("Usuario:", review['author']['steamid'])
    print("Comentario:", review['review'])
    print("Idioma:", review['language'])
    print("¿Recomienda?", review['voted_up'])
    print('-' * 40)

Usuario: 76561198079669556
Comentario: Para los que nunca lo han jugado y vinieron por recomendación o por el motivo que sea, no instalen esta m1erda de juego, por favor hagan caso a mi advertencia, tengo 10 años jugando esto, y dota ha arruinado mi vida, cuando creo que por fin dejé de jugarlo, algo o alguien o un pase de batalla, aparece de la nada y regresas por más, nunca es suficiente, nunca te llena, pierdes mas de lo que ganas y hay gente que merece algo peor que el infierno, este juego arruina la existencia misma, hay desesperación, agonía, desolación, peruanos, Techies, sufrimiento y un fuerte impulso por el su1c11dio, hermanos, por favor alejense de este juego y vivan sus vidas, enamorense, tengan hijos, viajen, coman, F0ll3n, vivan la vida, disfruten lo que nunca pude disfrutar, aun hay tiempo para ustedes, ya es tarde para mi.
Idioma: spanish
¿Recomienda? False
----------------------------------------
Usuario: 76561198288933432
Comentario: Un juego con excelsas mecanicas, p

De toda la información obtenida, tan solo nos interesa el texto de la reseña, el idioma de esta, y si el juego ha sido recomendado.

In [5]:
reviews = [review['review'] for review in data['reviews']]                # Obtener reviews a partir del diccionario
ratings = [review['voted_up'] for review in data['reviews']]              # Obterner ratings asociados a las reviews anteriores
idioma = [review['language'] for review in data['reviews']]               # Obtener idioma de las reviews

dataframe = pd.DataFrame({'reviews' : reviews ,  'idioma' : idioma , 'ratings' : ratings})     # Transformamos a formato pd.DataFrama para aprovechar sus múltiples utilidades
dataframe.head(10)

,reviews,idioma,ratings
0,Para los que nunca lo han jugado y vinieron po...,spanish,False
1,"Un juego con excelsas mecanicas, pero con la c...",spanish,False
2,"Es muy entretenido, si tienes horas libres pue...",spanish,True
3,"La verdad esque es muy buen juego, lo actualiz...",spanish,True
4,"después de 3 años siguiendo jugando, puedo dec...",spanish,True
5,"Sinceramente esta bien es un buen juego, pero ...",spanish,True
6,Continuacion del Dota clasico del Warcraft 3 p...,spanish,True
7,Buen juego pero la comunidad es un desastre va...,spanish,False
8,"bueno el dota 2 , es una moba que ace cambiar ...",spanish,True
9,oh.. solo hay que decir que una ves dentro j...,spanish,True


## Obtener una lista de juegos

Realizamos *Web Scraping* en la página de Steam, portal de venta de videojuegos, para obtener cuales son los juegos más jugados. De esta forma nos aseguramos que tenemos gran cantidad de comentarios en todos los juegos.

Para realizar el *Web Scraping* utilizamos Selenium ya que necesitamos interactuar con la página de Steam. Concretamente necesitamos scrollear para alcanzar el final de las página y así que cargue más juegos.

In [6]:
# Necesitamos selenium para hacer scroll en la página y cargar elementos

n = 9 # Número de scrolls
      # La página empieza con 50 juegos y carga 50 más por scroll

# Configurar Chrome
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Inicializar driver
driver = webdriver.Chrome(options=chrome_options)

# Cargar la página
driver.get("https://store.steampowered.com/search/?untags=9130&filter=topsellers&ndl=1")

# Realizar scrolls
for _ in range(n):
    # Scroll hasta el final de la página
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Esperar a que cargue contenido nuevo
    time.sleep(2)

# Obtener el HTML después de todos los scrolls
html = driver.page_source

# Cerrar el navegador
driver.quit()

Una vez descargado el html, lo tratamos con *BeautifulSoup* y extraemos los nombre de los juegos así como sus ID.

In [7]:
soup = BeautifulSoup(html, "html.parser")

tabla = soup.find_all("div", id="search_resultsRows")[0]

# Generamos una lista de diccionarios con la id y el título del juego.

# NOTA: Algunos de los "juegos más vendidos" no son juegos, como Steam Deck.
# Se puede filtrar más adelante, ya que en
# https://store.steampowered.com/api/appdetails?appids=1675200
# aparece como "type": "hardware", en vez de "type": "game" o "type": "dlc".
juegos = []

for game in tabla.find_all("a"):
  juegos.append({'id': game['data-ds-appid'], 'title': game.find("span", class_="title").text})

Comprovemos que hemos conseguido una lista adecuada.

In [8]:
juegos[:20]

[{'id': '1675200', 'title': 'Steam Deck'},
 {'id': '730', 'title': 'Counter-Strike 2'},
 {'id': '1466060', 'title': 'Tainted Grail: The Fall of Avalon'},
 {'id': '570', 'title': 'Dota 2'},
 {'id': '553850', 'title': 'HELLDIVERS™ 2'},
 {'id': '2104890', 'title': 'RoadCraft'},
 {'id': '2622380', 'title': 'ELDEN RING NIGHTREIGN'},
 {'id': '2993780', 'title': 'FANTASY LIFE i: The Girl Who Steals Time'},
 {'id': '1142710', 'title': 'Total War: WARHAMMER III'},
 {'id': '2183900', 'title': 'Warhammer 40,000: Space Marine 2'},
 {'id': '2186680', 'title': 'Warhammer 40,000: Rogue Trader'},
 {'id': '1091500', 'title': 'Cyberpunk 2077'},
 {'id': '230410', 'title': 'Warframe'},
 {'id': '2767030', 'title': 'Marvel Rivals'},
 {'id': '2669320', 'title': 'EA SPORTS FC™ 25'},
 {'id': '1903340', 'title': 'Clair Obscur: Expedition 33'},
 {'id': '1086940', 'title': "Baldur's Gate 3"},
 {'id': '1174180', 'title': 'Red Dead Redemption 2'},
 {'id': '3164500', 'title': 'Schedule I'},
 {'id': '3059520', 'title

## Extracción de reseñas

Una vez ya tenemos una lista de juegos con Ids, pasamos a la extracción de reseñas de cada juego.

In [9]:
rev_por_juego = 100
all_reviews = []

for juego in juegos:
    game_id = juego['id']
    game_title = juego['title']
    url = f'https://store.steampowered.com/appreviews/{game_id}?json=1&num_per_page={rev_por_juego}&language=spanish'
    try:
        response = requests.get(url)
        data = response.json()
        for review in data.get('reviews', []):
            all_reviews.append({
                'game_title': game_title,
                'game_id': game_id,
                'review': review['review'],
                'language': review['language'],
                'recommended': review['voted_up']
            })
    except Exception as e:
        print(f"Error loading reviews for {game_title} ({game_id}): {e}")

reviews_df = pd.DataFrame(all_reviews)

En el dataset obtenido tenemos:
* **game_title**: Título del juego.
* **game_id**: Identificador del juego.
* **review**: La review.
* **recommended**: Si se ha recomendado o no el juego.

In [10]:
reviews_df.head()

,game_title,game_id,review,language,recommended
0,Counter-Strike 2,730,juego de mierd4 te odio si un dia un amigo les...,spanish,True
1,Counter-Strike 2,730,"Buen juego, es sorprendente la cantidad de ins...",spanish,True
2,Counter-Strike 2,730,This game its imposible to play. Its full of c...,spanish,False
3,Counter-Strike 2,730,"Este juego del diablo te deja: sin plata, sin ...",spanish,True
4,Counter-Strike 2,730,"El juego esta bien, el tema son los hackers qu...",spanish,True


# Procesado del texto

Si bien ya hemos obtenido una gran cantidad de reviews, cerrando el capítulo de *web scrapping*, ahora debemos preparar el texto para poder trabajar con este.

## Filtro por idioma

Tan solo queremos trabajar con reviews en español, así que filtramos por el idioma en el que está escrito cada review.

In [11]:
reseñas_df = reviews_df[reviews_df['language'] == 'spanish'].reset_index(drop=True)
reseñas_df.head(10)

,game_title,game_id,review,language,recommended
0,Counter-Strike 2,730,juego de mierd4 te odio si un dia un amigo les...,spanish,True
1,Counter-Strike 2,730,"Buen juego, es sorprendente la cantidad de ins...",spanish,True
2,Counter-Strike 2,730,This game its imposible to play. Its full of c...,spanish,False
3,Counter-Strike 2,730,"Este juego del diablo te deja: sin plata, sin ...",spanish,True
4,Counter-Strike 2,730,"El juego esta bien, el tema son los hackers qu...",spanish,True
5,Counter-Strike 2,730,"10/10, racismo y degeneración a raudales. Si o...",spanish,True
6,Counter-Strike 2,730,"la comunidad es un mierda, las mecanicas son a...",spanish,True
7,Counter-Strike 2,730,Un juego increíble al igual que lo que se la s...,spanish,True
8,Counter-Strike 2,730,"me dio un derrame cerebral mientras jugaba, me...",spanish,False
9,Counter-Strike 2,730,"Es un juegazo, lastima la cantidad de chiteros...",spanish,False


## Detección y eliminación de arte

Es común que los usuarios de Steam incluyan en sus reseñas dibujos realizados mediante metacarácteres.

In [12]:
# Función para contar metacaracteres y alfanuméricos
def more_meta_than_alnum(text):
    if not isinstance(text, str):
        return False
    meta = re.findall(r'\W', text, re.UNICODE)
    alnum = re.findall(r'\w', text, re.UNICODE)
    return len(meta) > len(alnum)

# Filtra y muestra las reviews que cumplen la condición
reviews_meta = reseñas_df[reseñas_df['review'].apply(more_meta_than_alnum)]
print(reviews_meta['review'].iloc[9])

(Y)


Dado que estas reseñas pueden darnos problemas, las identificaremos y las eliminaremos de nuestro dataset.

In [13]:
# Elimina reviews vacías o solo espacios
reseñas_df = reseñas_df[reseñas_df['review'].notnull() & (reseñas_df['review'].str.strip() != '')]

# Elimina reviews con más metacaracteres que alfanuméricos
reseñas_df = reseñas_df[~reseñas_df['review'].apply(more_meta_than_alnum)].reset_index(drop=True)

## Limpieza y normalización del texto

El siguiente paso, muy necesario en nuestro contexto, es la corrección ortográfica: las reseñas de Steam muchas veces contienen _slangs_, insultos y un tono informal, lo cual dificulta cualquier tipo de clasificación. En este caso, haremos uso de la librería **spellchecker**:

In [16]:
nlp=spacy.load('es_core_news_md')
stop_words = get_stop_words('spanish') + ['ser', 'cosa', '#']

pattern2 = re.compile('[{}]'.format(re.escape(string.punctuation))) #selecciona símbolos de puntuación

def clean_text(text):
    """Convertimos en tokens,
    eliminamos los tokens que son signos de puntuación, convertimos en
    minúsculas y quitamos signos de puntuación. Para terminar
    volvemos a convertir en cadena de texto"""
    tokens = nlp(text)
    tokens = [tok.lower_ for tok in tokens if not tok.is_punct and not tok.is_space]
    filtered_tokens = [pattern2.sub('', token) for token in tokens if not (token in stop_words)] #obvia stop_words y después quita signos de puntuación
    filtered_text = ' '.join(filtered_tokens)
    filtered_text = unidecode(filtered_text) # Elimina acentos
    return filtered_text

def lemmatize_text(text):
    """Convertimos el texto a tokens, extraemos el lema de cada token
    y volvemos a convertir en cadena de texto sin acentos"""
    tokens = nlp(text)
    lemmatized_tokens = [tok.lemma_ for tok in tokens]
    lemmatized_text = ' '.join(lemmatized_tokens)
    deaccented_str = unidecode(lemmatized_text)

    return deaccented_str

def remove_stopwords(text, stop_words = stop_words):
    """Tokeniza el texto, elimina las stopwords y devuelve la cadena sin ellas."""
    tokens = nlp(text)
    tokens = [tok.lower_ for tok in tokens if not tok.is_punct and not tok.is_space]
    filtered_tokens = [pattern2.sub('', token) for token in tokens]
    words = filtered_tokens

    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_text = ' '.join(filtered_words)
    return filtered_text

def safe_detect(text):
    try:
        # Convertir a string y manejar valores nulos
        if pd.isnull(text):
            return 'unknown'
            
        return detect(text)
    except LangDetectException:
        return 'unknown'


In [17]:
# Aplicamos un segundo filtro de idioma (la label de steam no es 100% fiable)
reseñas_df = reseñas_df[ reseñas_df['review'].apply(safe_detect) == 'es'].copy()

# Añade columna con el texto sin stopwords
reseñas_df['no_stopwords'] = reseñas_df['review'].apply(remove_stopwords)

# Añade columna con el texto limpio
reseñas_df['clean_text'] = reseñas_df['review'].apply(clean_text)

# Añade columna con la lematización del texto limpio
reseñas_df['lemmatized_text'] = reseñas_df['clean_text'].apply(lemmatize_text)

Hemos aplicado así un preprocesado estándar para cualquier clase de textos escritos en español, pero nuestro caso de estudio es mucho más concreto, y por tanto podemos esperar repeticiones elevadas de palabras como juego o jugar, que realmente no aportan ningún tipo de información para el objetivo de clasificación. Consecuentemente parece razonable suponer que eliminar estas palabras ayudará al desempeño del modelo. Así, obtenemos una lista de las 20 palabras más frecuentes:

In [18]:
# Convertir todos los valores a string y reemplazar NaNs por cadenas vacías
texts = reseñas_df['lemmatized_text'].fillna('').astype(str)

# Procesar con spaCy
docs = list(nlp.pipe(texts))

# Extraer todas las palabras (tokens) de todos los textos
all_words = []
for doc in docs:
    all_words.extend([token.text for token in doc if not token.is_space])

# Convertir a array de numpy
all_words_array = np.array(all_words)

# Usar pandas para contar frecuencia
freq_series = pd.Series(all_words_array).value_counts().reset_index()
freq_series.columns = ['word', 'freq']
print(freq_series[0:20])

        word  freq
0      juego  8778
1         si  3308
2      jugar  2470
3      poder  2406
4         el  2383
5      hacer  2263
6      mejor  1982
7       buen  1905
8        mas  1683
9         ir  1403
10      solo  1275
11     bueno  1239
12      bien  1212
13      hora  1067
14      cada  1007
15       vez   979
16  historia   920
17       dar   915
18       ver   884
19    gustar   882


Y confirmamos nuestras sospechas, con lo que aplicamos de nuevo un fitlrado de stopwords personalizado a la columna de texto lematizado.

In [19]:
additional_stopwords = [word for word in list(freq_series['word'][0:20]) if word not in ['bien', 'bueno', 'gustar', 'dar', 'mejor']]

# Comprobamos las stopwords adicionales
print(additional_stopwords)

# Añadimos a la lista original
videogames_stopwords = stop_words + additional_stopwords

# Añade columna con la lematización del texto filtrada por stopwords propias de videojuegos
reseñas_df['lemmatized_text_v2'] = reseñas_df['lemmatized_text'].apply(lambda text: remove_stopwords(text, stop_words = videogames_stopwords))

['juego', 'si', 'jugar', 'poder', 'el', 'hacer', 'buen', 'mas', 'ir', 'solo', 'hora', 'cada', 'vez', 'historia', 'ver']


Comprobamos ahora cuáles son las palabras más frecuentes

In [20]:
# Convertir todos los valores a string y reemplazar NaNs por cadenas vacías
texts = reseñas_df['lemmatized_text_v2'].fillna('').astype(str)

# Procesar con spaCy
docs = list(nlp.pipe(texts))

# Extraer todas las palabras (tokens) de todos los textos
all_words = []
for doc in docs:
    all_words.extend([token.text for token in doc if not token.is_space])

# Convertir a array de numpy
all_words_array = np.array(all_words)

# Usar pandas para contar frecuencia
freq_series = pd.Series(all_words_array).value_counts().reset_index()
freq_series.columns = ['word', 'freq']
print(freq_series[0:20])

           word  freq
0         mejor  1982
1         bueno  1239
2          bien  1212
3           dar   915
4        gustar   882
5         decir   735
6     divertido   721
7         amigo   695
8         pasar   686
9         mundo   677
10       querer   672
11    personaje   661
12     bastante   647
13      primero   643
14  entretenido   642
15        nuevo   628
16          asi   625
17      grafico   590
18       tiempo   589
19         vida   582


In [21]:
# Reordena las columnas para que 'clean_text' y 'lemmatized_text' estén después de 'review'
columnas = list(reseñas_df.columns)
columnas.remove('language')

# Reordena las columnas
nueva_orden = columnas.copy()
if 'review' in columnas and 'clean_text' in columnas and 'lemmatized_text' and 'lemmatized_text_v2' in columnas in columnas:
    idx = columnas.index('review')
    # Quita las columnas para reinsertarlas en orden correcto
    nueva_orden.remove('clean_text')
    nueva_orden.remove('lemmatized_text')
    nueva_orden.remove('lemmatized_text_v2')
    nueva_orden = (
        nueva_orden[:idx+1] +
        ['clean_text', 'lemmatized_text', 'lemmatized_text_v2'] +
        nueva_orden[idx+1:]
    )

reseñas_df = reseñas_df[nueva_orden]

In [22]:
reseñas_df.head(10) 

,game_title,game_id,review,recommended,no_stopwords,clean_text,lemmatized_text,lemmatized_text_v2
0,Counter-Strike 2,730,juego de mierd4 te odio si un dia un amigo les...,True,juego mierd4 odio si dia amigo dice jueguen di...,juego mierd4 odio si dia amigo dice jueguen di...,juego mierd4 odiar si dia amigo decir jugar di...,mierd4 odiar dia amigo decir directamente bloq...
1,Counter-Strike 2,730,"Buen juego, es sorprendente la cantidad de ins...",True,buen juego sorprendente cantidad insultos crea...,buen juego sorprendente cantidad insultos crea...,buen juego sorprendente cantidad insulto creat...,sorprendente cantidad insulto creativo encontr...
3,Counter-Strike 2,730,"Este juego del diablo te deja: sin plata, sin ...",True,juego diablo deja plata amigos pareja futuro p...,juego diablo deja plata amigos pareja futuro p...,juego diablo dejar plata amigo parejo futuro p...,diablo dejar plata amigo parejo futuro platar ...
4,Counter-Strike 2,730,"El juego esta bien, el tema son los hackers qu...",True,juego bien tema hackers joden incluso premier ...,juego bien tema hackers joden incluso premier ...,juego bien tema hackers joden incluso premier ...,bien tema hackers joden incluso premier empare...
5,Counter-Strike 2,730,"10/10, racismo y degeneración a raudales. Si o...",True,1010 racismo degeneración raudales si odias vi...,1010 racismo degeneracion raudales si odias vi...,1010 racismo degeneracion raudal si odiar vida...,1010 racismo degeneracion raudal odiar vida pr...
6,Counter-Strike 2,730,"la comunidad es un mierda, las mecanicas son a...",True,comunidad mierda mecanicas complicados si paga...,comunidad mierda mecanicas complicados si paga...,comunidad mierda mecanica complicado si pagar ...,comunidad mierda mecanica complicado pagar pre...
7,Counter-Strike 2,730,Un juego increíble al igual que lo que se la s...,True,juego increíble igual suda valve solo buscan o...,juego increible igual suda valve solo buscan o...,juego increible igual sudar valve solo buscar ...,increible igual sudar valve buscar obtener din...
8,Counter-Strike 2,730,"me dio un derrame cerebral mientras jugaba, me...",False,dio derrame cerebral mientras jugaba arrepient...,dio derrame cerebral mientras jugaba arrepient...,dar derrame cerebral mientras jugar arrepienti...,dar derrame cerebral mientras arrepientir habe...
9,Counter-Strike 2,730,"Es un juegazo, lastima la cantidad de chiteros...",False,juegazo lastima cantidad chiteros competivo pr...,juegazo lastima cantidad chiteros competivo pr...,juegazo lastimo cantidad chitero competivo pre...,juegazo lastimo cantidad chitero competivo pre...
10,Counter-Strike 2,730,"una re mil mierda, no saben actualizar nada si...",False,re mil mierda saben actualizar romper medio ju...,re mil mierda saben actualizar romper medio ju...,re mil mierda saber actualizar romper medio ju...,re mil mierda saber actualizar romper medio en...


## Exportación del dataset

Una vez preparado el datset, lo guardamos en formato .csv para el uso con modelos.

In [24]:
reseñas_df.to_csv('steam_reseñas.csv', index=False, encoding='utf-8')

## Extracción de la información de los juegos

Obtenemos mediante web scraping información en español de los juegos de los cuales hemos sacado los comentarios: un resumen del juego, los géneros y las categorias del juego.

In [20]:
ids = [game['id'] for game in juegos]
game_description_list = []
short_description_list = []
genres_list = []
categories_list = []

for id in ids:
  url = 'https://store.steampowered.com/api/appdetails?appids=' + id + '&l=spanish'

  # Realiza la petición
  response_2 = requests.get(url)
  data_2 = response_2.json()

  game_description_list.append(re.sub(r'<.+?>', ' ', data_2[id]['data']['detailed_description']))

  if 'short_description' in data_2[id]['data'].keys():
    short_description_list.append(re.sub(r'<.+?>', ' ', data_2[id]['data']['short_description']))

  genres = []
  if 'genres' in data_2[id]['data'].keys():
    for genre in data_2[id]['data']['genres']:
      genres.append(genre['description'])
  genres_list.append(genres)

  categories = []
  if 'categories' in data_2[id]['data'].keys():
    for category in data_2[id]['data']['categories']:
      categories.append(category['description'])
  categories_list.append(categories)

  time.sleep(2)

Guardando en un dataset la información:
* **ids**: Identificador del juego.
* **game_description**: Resumen del juego.
* **short_description**: Resumen corto del juego.
* **genres**: Géneros del juego.
* **categories**: Categorías del juego.

Incluimos el **short_description** además del **game_description**.

Esto es debido a que, aunque **short_description** sea, como su nombre indica, más corto; este siempre contendrá texto, al contrario que **game_description**, el cual, en ocasiones, solo contiene imágenes y gifs.

In [21]:
game_description_df = pd.DataFrame({'game_id' : ids,
                            'game_description' : game_description_list,
                            'short_description' : short_description_list,
                            'genres' : genres_list,
                            'categories' : categories_list})

In [22]:
# Añade columna con el resumen del juego limpio
game_description_df['clean_text'] = game_description_df['game_description'].apply(clean_text)

# Añade columna con la lematización del resumen del juego limpio
game_description_df['lemmatized_text'] = game_description_df['clean_text'].apply(lemmatize_text)

# Realizamos el mismo proceso con el resumen corto del juego
game_description_df['clean_short_text'] = game_description_df['short_description'].apply(clean_text)
game_description_df['lemmatized_short_text'] = game_description_df['clean_short_text'].apply(lemmatize_text)

In [23]:
game_description_df.head(10)

,game_id,game_description,short_description,genres,categories,clean_text,lemmatized_text,clean_short_text,lemmatized_short_text
0,1675200,,,[],[],,,,
1,730,"Durante las dos últimas décadas, Counter‑Strik...","Durante las dos últimas décadas, Counter‑Strik...","[Acción, Free to Play]","[Multijugador, Multijugador multiplataforma, C...",dos ultimas decadas counter-strike proporciona...,dos ultima decada counter-strike proporcionado...,dos ultimas decadas counter-strike proporciona...,dos ultima decada counter-strike proporcionado...
2,1466060,TAINTED GRAIL: THE FALL OF AVALON - SUPPORTER...,Adéntrate en una oscura reinterpretación de la...,"[Acción, Aventura, Indie, Rol]","[Un jugador, Compat. total con mando, Steam Cl...",tainted grail the fall of avalon supporter edi...,tainted grail the fall of avalon supporter edi...,adentrate oscura reinterpretacion leyendas art...,adentrate oscuro reinterpretacion leyenda artu...
3,570,"El juego más jugado en Steam . Cada día, mill...","Cada día, millones de jugadores de todo el mun...","[Acción, Estrategia, Free to Play]","[Multijugador, Cooperativo, Cromos de Steam, S...",juego jugado steam cada dia millones jugadores...,juego jugado steam cada dia millon jugador mun...,cada dia millones jugadores mundo entran batal...,cada dia millon jugador mundo entrar batalla c...
4,553850,Digital Deluxe Edition La edición incluye...,La última línea de ataque de la galaxia. Alíst...,[Acción],"[Multijugador, Cooperativo, Cooperativo en lín...",digital deluxe edition edicion incluye conjunt...,digital deluxe edition edicion incluir conjunt...,ultima linea ataque galaxia alistate helldiver...,ultimo linea ataque galaxia alistate helldiver...
5,2104890,REBUILD EDITION Consigue la Edición Rebu...,Diriges una empresa que restaura lugares devas...,"[Aventura, Simuladores]","[Un jugador, Multijugador, Cooperativo, Cooper...",rebuild edition consigue edicion rebuild roadc...,rebuild edition conseguir edicion rebuild road...,diriges empresa restaura lugares devastados de...,diriges empresa restaura lugar devastado desas...
6,2622380,Deluxe Edition La edición Deluxe incluye: ...,ELDEN RING NIGHTREIGN es una aventura independ...,"[Acción, Rol]","[Un jugador, Multijugador, Cooperativo, Cooper...",deluxe edition edicion deluxe incluye elden ri...,deluxe edition edicion deluxe incluir eldir ri...,elden ring nightreign aventura independiente d...,eldir ring nightreign aventura independiente d...
7,2993780,Adéntrate en este juego RPG de vida tranqui...,Adéntrate en este juego RPG de vida tranquila ...,[Rol],"[Un jugador, Multijugador, Cooperativo, Cooper...",adentrate juego rpg vida tranquila podras camb...,adentrate juego rpg vida tranquilo podra cambi...,adentrate juego rpg vida tranquila podras camb...,adentrate juego rpg vida tranquilo podra cambi...
8,2767030,Un elenco de Marvel variado y detallado ...,¡Marvel Rivals es un juego de disparos de JcJ ...,"[Acción, Free to Play]","[Multijugador, JcJ, JcJ en línea, Cooperativo,...",elenco marvel variado detallado elige amplia v...,elenco marvel variado detallado elegir amplio ...,marvel rivals juego disparos jcj equipos super...,marvel rivals juego dispar jcj equipo superher...
9,230410,Despierta como un guerrero imparable y lucha j...,Despierta como un guerrero imparable y lucha j...,"[Acción, Rol, Free to Play]","[Un jugador, Multijugador, Cooperativo, Cooper...",despierta guerrero imparable lucha junto amigo...,despertar guerrero imparable luchar junto amig...,despierta guerrero imparable lucha junto amigo...,despertar guerrero imparable luchar junto amig...


## Exportación del dataset

Al igual que con las reseñas, guardamos el dataset en formato .csv para usarlas con los modelos.

In [24]:
game_description_df.to_csv('steam_descripciones_juegos.csv', index=False, encoding='utf-8')

# Data augmentation

Vamos a intentar paliar el problema del desbalanceo de clase generando datos sintéticos de la clase menos frecuente (No recomendado).

In [25]:
reseñas_negativas = reseñas_df[reseñas_df['recommended'] == True]